# Toronto Neighborhoods Clustering

Dependencies download:

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

Install and import BeautifulSoup:

In [3]:
#!conda install -c conda-forge python3-bs4 --yes
from bs4 import BeautifulSoup # HTML scraping library

print('BeautifulSoup imported.')

BeautifulSoup imported.


Download Wiki page:

In [4]:
!wget -q -O 'toronto_wiki.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


Load data:

In [4]:
with open('toronto_wiki.html') as html_doc:
    toronto_soup = BeautifulSoup(html_doc, 'html.parser')

In [5]:
neib_table = toronto_soup.find_all("table", limit=1)[0]
rows = neib_table.find_all("tr")


#### Tranform the data into a *pandas* dataframe

Define pandas table columns:

In [20]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 



Create and fill dataframe:

In [22]:
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for tr in rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    if (len(row) > 0):
        if (row[1] != 'Not assigned'):
            postal_code = row[0]
            borough = row[1]
            neighborhood_name = row[2][:-1]
            if (neighborhood_name == 'Not assigned'):
                neighborhood_name = borough

            neighborhoods = neighborhoods.append({
                'PostalCode': postal_code,
                'Borough': borough,
                'Neighborhood': neighborhood_name}, ignore_index=True)
            
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Group neighborhoods by postal codes:

In [23]:
neighborhoods_by_postal = neighborhoods.groupby(['PostalCode', 'Borough'], as_index=False, sort=False).agg(lambda x: (',').join(x))
neighborhoods_by_postal.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Save dataframe for later use:

In [25]:
neighborhoods_by_postal.to_csv('neighborhoods_by_postal.csv')

Get number of rows:

In [26]:
print("Rows: ", neighborhoods_by_postal.shape[0])

Rows:  103
